In [32]:
#CSV 파일을 불러올 때 Schema를 추정할 수 있다. 애당초 Header가 잘 돼 있으면 헤더만으로도 충분한 듯.
flightData2015=spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv("/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv")

In [63]:
#잘 가져왔나 찍어보자.
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [64]:
#count 값을 기준으로 정렬한다.
flightData2015.sort("count")

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

In [65]:
#파티션 수를 5개로 줄여보자. 어차피 콩알만한 데이터라 금방 끝난다.
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [66]:
#정렬이 어떤 식으로 수행되는지 확인해보자. 파일을 이러이러한 조건 걸고 스캔-설정된 파티션 수대로 일하기로 했다-정렬한다
flightData2015.sort("count").explain()

== Physical Plan ==
*(2) Sort [count#102 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#102 ASC NULLS FIRST, 5)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#100,ORIGIN_COUNTRY_NAME#101,count#102] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [68]:
#정렬된 데이터를 찍어보자. count가 낮은 순서로 나온다.
flightData2015.sort("count").take(3)

[Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1)]

In [69]:
#읽어온 데이터로 테이블을 만들자!
flightData2015.createOrReplaceTempView("flight_data_2015")

In [70]:
#이제 sql로 질의를 날릴 수 있다. 
sqlWay=spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

In [71]:
#근데 사실 저렇게 테이블 안 만들어도 이런건 그대로 할 수 있다. 
dataFrameWay=flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

In [72]:
#테이블에서 데이터를 어떻게 가져오나 본다.
sqlWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#100], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#100, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#100], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#100] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [73]:
#테이블을 만들지 않은 경우에는 데이터를 어떻게 가져오나 본다. 근데 똑같음. 그러니까 성능상으로도 차이는 없나보다.
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#100], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#100, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#100], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#100] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [74]:
#제일 큰 값을 가져오는 max()다. sql 사용할 때는 그냥 이렇게 하면 된다.
spark.sql("SELECT max(count) from flight_data_2015").take(1)

[Row(max(count)=370002)]

In [75]:
#어떻게 돌아가는지 실행 계획을 까본다.
spark.sql("SELECT max(count) from flight_data_2015").explain()

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[max(count#102)])
+- Exchange SinglePartition
   +- *(1) HashAggregate(keys=[], functions=[partial_max(count#102)])
      +- *(1) FileScan csv [count#102] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<count:int>


In [76]:
#sql 안 쓰면 이렇게 import 해줘야 한다.
from pyspark.sql.functions import max

In [77]:
#마찬가지로 max를 날려보자. 결과는 당연히 같다.
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [78]:
#얘는 동작 방식이 다른가 까보자. 똑같다!
flightData2015.select(max("count")).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[max(count#102)])
+- Exchange SinglePartition
   +- *(1) HashAggregate(keys=[], functions=[partial_max(count#102)])
      +- *(1) FileScan csv [count#102] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<count:int>


In [82]:
#각 나라들이 목적지로 쓰인 횟수를 집계한다. sql로. destination_total 기준으로 내림차순 정렬함.
maxSql=spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

In [87]:
#출력한다. 실행계획도 까보자.
maxSql.show()
maxSql.explain()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[aggOrder#284L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#100,destination_total#282L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#100], functions=[sum(cast(count#102 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#100, 5)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#100], functions=[partial_sum(cast(count#102 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#100,count#102] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilters: [], 

In [84]:
#이번에는 sql 안 쓰고 같은 데이터를 요청해봅니다. 일단 내림차순은 import 해야함.
from pyspark.sql.functions import desc

In [89]:
#sql 질의문보다 복잡해보인다! 귀찮지만 실행한다. 같은 결과가 나온다. 
#실행계획을 읽어보면 파일 스캔-파티션-destination_total로 내림차순 정렬-limit 5개만 꺼내다 출력
flightData2015.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

flightData2015.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.explain()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#427L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#100,destination_total#427L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#100], functions=[sum(cast(count#102 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#100, 5)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#100], functions=[partial_sum(cast(count#102 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#100,count#102] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilt